<a href="https://colab.research.google.com/github/brittabeta/Data-Extraction/blob/main/e_commerce_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Data from an Online Store

*note: displayed customers and products are fictional*

# Prepare 

In [61]:
!pip install --upgrade ShopifyAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ShopifyAPI: filename=ShopifyAPI-12.3.0-py3-none-any.whl size=46595 sha256=ed36380937c98e3b16764019a1499dfd5d388332ce90ba25b5bd5a38ea9b7e72
  Stored in directory: /root/.cache/pip/wheels/b3/cc/5c/13a4c4e82a9d184a650e3df28b701ef27ea95c7258f54e1a22
  Created wheel for pyactiveresource: filename=pyactiveresource-2.2.2-py3-none-any.whl size=23324 sha256=a898d0d27bb48ad21acba3a3d9fbd8864ef3d43b055fe10a1bdeb2317e6d4ff6
  Stored in directory: /root/.cache/pip/wheels/48/28/af/08c9f6f84965d3eef47d6d468accd89769fd198f257ea84fd9
Successfully built ShopifyAPI pyactiveresource


In [55]:
# assign api secrets to variables
f = '/content/shopify970.json' # file path

df = pd.read_json(f, orient='index') # index because file is dict like

token = df.loc['API_KEY', 0]
str(token) # to add quotes to variable

shop = df.loc['endpoint', 0]
str(shop)

merchant = 'store970.myshopify.com'

print('secrets ready') # prevent print of secrets

secrets ready


In [62]:
# import tools
import pandas as pd
import shopify
import urllib.request
import json
from IPython.utils.text import strip_email_quotes

# Contract and Connect

In [72]:
# define a function to create a connection and authenticate requests
def shopify_client():
  api_session = shopify.Session(merchant, '2023-04', token) # version see https://shopify.dev/docs/api/admin-graphql#authentication
  shopify.ShopifyResource.activate_session(api_session)
  client = shopify.GraphQL()
  return client

In [104]:
# assign queries to variables
products = """
query {
  productVariants (first: 20) {
    edges {
      node {
        id
        displayName
        title
        sku
        inventoryQuantity
        price
        inventoryItem {
          requiresShipping
        }
      }
    }
  }
  }
  """

In [154]:
customers = """
query {
  customers (first: 20) {
    edges {
      node {
        id
        firstName
        lastName
        email
        locale
        createdAt
        updatedAt
        numberOfOrders
        amountSpent {
          amount
          currencyCode
        }
      }
    }
  }
}
"""

# Query the Store: Extract Data

In [96]:
# open session and authentication
client = shopify_client()
# request data from store, argument=query
data = json.loads(client.execute(customers))
data

{'data': {'customers': {'edges': [{'node': {'id': 'gid://shopify/Customer/6864981721377',
      'firstName': 'Ayumu',
      'lastName': 'Hirano',
      'email': 'ayumu.hirano@example.com',
      'locale': 'en',
      'createdAt': '2023-03-10T18:11:14Z',
      'updatedAt': '2023-03-10T18:11:21Z',
      'numberOfOrders': '2',
      'amountSpent': {'amount': '2859.8', 'currencyCode': 'USD'}}},
    {'node': {'id': 'gid://shopify/Customer/6864981754145',
      'firstName': 'Russell',
      'lastName': 'Winfield',
      'email': 'russel.winfield@example.com',
      'locale': 'en',
      'createdAt': '2023-03-10T18:11:14Z',
      'updatedAt': '2023-03-10T18:11:23Z',
      'numberOfOrders': '4',
      'amountSpent': {'amount': '6656.19', 'currencyCode': 'USD'}}},
    {'node': {'id': 'gid://shopify/Customer/6864981786913',
      'firstName': 'Karine',
      'lastName': 'Ruby',
      'email': 'karine.ruby@example.com',
      'locale': 'en',
      'createdAt': '2023-03-10T18:11:14Z',
      'updat

In [106]:
# request another query
data2 =  json.loads(client.execute(products))
data2

{'data': {'productVariants': {'edges': [{'node': {'id': 'gid://shopify/ProductVariant/44692793524513',
      'displayName': 'a - S',
      'title': 'S',
      'sku': 'A123',
      'inventoryQuantity': 20,
      'price': '1.00',
      'inventoryItem': {'requiresShipping': True}}},
    {'node': {'id': 'gid://shopify/ProductVariant/44692793557281',
      'displayName': 'a - M',
      'title': 'M',
      'sku': 'A124',
      'inventoryQuantity': 11,
      'price': '1.00',
      'inventoryItem': {'requiresShipping': True}}},
    {'node': {'id': 'gid://shopify/ProductVariant/44692793590049',
      'displayName': 'a - L',
      'title': 'L',
      'sku': 'A125',
      'inventoryQuantity': 11,
      'price': '1.00',
      'inventoryItem': {'requiresShipping': True}}},
    {'node': {'id': 'gid://shopify/ProductVariant/44692812661025',
      'displayName': 'b - Yellow',
      'title': 'Yellow',
      'sku': 'B223',
      'inventoryQuantity': 40,
      'price': '2.00',
      'inventoryItem': {'re

### Bulk Query for Large Requests

In [161]:
# running a bulk operation mutation for large query
def bulk_operation(query):
  q="""
  mutation {{
    bulkOperationRunQuery(
  query:\"""{query}\"""
    ) {{
      bulkOperation {{
        id
        status
        }}
        userErrors {{
          field
          message
         }}
       }}
     }}
      """.format(query=query)
  return q
  # query to check status 
  q_bulk_status="""
  query bulkStatus($id: ID!) {
    node (id: $id) {
      ... on BulkOperation {
        id
        status
        errorCode
        createdAt
        completedAt
        objectCount
        fileSize
        url
        partialDataUrl
      }
    }
  }
  """

In [162]:
  # query to check status 
q_bulk_status="""
  query bulkStatus($id: ID!) {
    node (id: $id) {
      ... on BulkOperation {
        id
        status
        errorCode
        createdAt
        completedAt
        objectCount
        fileSize
        url
        partialDataUrl
      }
    }
  }
  """

In [163]:
def bulk_status(client,bulk):
  bulk_id=json.loads(bulk)['data']['bulkOperationRunQuery']['bulkOperation']['id']
  status=json.loads(client.execute(q_bulk_status,{'id':bulk_id}))
  return status

In [165]:
bulk_query = bulk_operation(customers)
bulk = client.execute(bulk_query)
bulk # but wait...see more

'{"data":{"bulkOperationRunQuery":{"bulkOperation":{"id":"gid://shopify/BulkOperation/3210129244449","status":"CREATED"},"userErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":1000.0,"currentlyAvailable":990,"restoreRate":50.0}}}}'

In [160]:
status=bulk_status(client,bulk)
status

{'data': {'node': {'id': 'gid://shopify/BulkOperation/3210121937185',
   'status': 'COMPLETED',
   'errorCode': None,
   'createdAt': '2023-05-24T17:45:50Z',
   'completedAt': '2023-05-24T17:45:50Z',
   'objectCount': '3',
   'fileSize': '861',
   'url': 'https://storage.googleapis.com/shopify-tiers-assets-prod-us-east1/6nbwx6dz1jv3fk1ek8d5a1wyyq6h?GoogleAccessId=assets-us-prod%40shopify-tiers.iam.gserviceaccount.com&Expires=1685555150&Signature=epz5oLb8s9ZlmttLAMoCfp42XpPNQsiSLUwvwC3JTVUNNq%2FWHQYAsubm9zWzMgynQ3EhqyOVWdXDMHAWRI9bkjj19bNk2i86QXmOfCB%2BqX4%2BvmxrIHnVj2vitKAD3A3AczzKtVcownuCkNh7vmHjzDU1DwZMjS1A%2FwCfwxzrcvgP2b85PBvlbyPTAL6HR0bifwz6%2BGKjSdlY9ZSxGYGrpn4wawTTEztcmlrc6zJKtIzVLyYRzvphJV8le7GaziksPemz%2B%2BPHGgT38fplsrulKj339ICG2LtDI238mIeZSzoxqxIShMx%2BpBpx5Ra7S8AS2WYrskPAJfSLFFtZKmsPMw%3D%3D&response-content-disposition=attachment%3B+filename%3D%22bulk-3210121937185.jsonl%22%3B+filename%2A%3DUTF-8%27%27bulk-3210121937185.jsonl&response-content-type=application%2Fjsonl',
   

fetch data from url:
```
{"id":"gid:\/\/shopify\/Customer\/6864981721377","firstName":"Ayumu","lastName":"Hirano","email":"ayumu.hirano@example.com","locale":"en","createdAt":"2023-03-10T18:11:14Z","updatedAt":"2023-03-10T18:11:21Z","numberOfOrders":"2","amountSpent":{"amount":"2859.8","currencyCode":"USD"}}
{"id":"gid:\/\/shopify\/Customer\/6864981754145","firstName":"Russell","lastName":"Winfield","email":"russel.winfield@example.com","locale":"en","createdAt":"2023-03-10T18:11:14Z","updatedAt":"2023-03-10T18:11:23Z","numberOfOrders":"4","amountSpent":{"amount":"6656.19","currencyCode":"USD"}}
{"id":"gid:\/\/shopify\/Customer\/6864981786913","firstName":"Karine","lastName":"Ruby","email":"karine.ruby@example.com","locale":"en","createdAt":"2023-03-10T18:11:14Z","updatedAt":"2023-03-10T18:11:23Z","numberOfOrders":"1","amountSpent":{"amount":"752.18","currencyCode":"USD"}}
```

# Clean and Create

In [238]:
# or fetch data from url with below
# clean and create dataframe
def fetch_bulk(bulk_status):
  b = status['data']['node']['url']
  bulk_df = pd.read_json(b, lines=True)
  amount_ = []
  currency_code = []
  for i in list(bulk_df.index): 
    a = float(bulk_df['amountSpent'][i]['amount'])
    b = strip_email_quotes(bulk_df['amountSpent'][i]['currencyCode'])
    amount_.append(a)
    currency_code.append(b)
  bulk_df['amount'] = amount_
  bulk_df['currency_code'] = currency_code
  bulk_df.drop('amountSpent', axis=1, inplace=True)
  return bulk_df

In [239]:
fetch_bulk(status)

,id,firstName,lastName,email,locale,createdAt,updatedAt,numberOfOrders,amount,currency_code
0,gid://shopify/Customer/6864981721377,Ayumu,Hirano,ayumu.hirano@example.com,en,2023-03-10T18:11:14Z,2023-03-10T18:11:21Z,2,2859.80,USD
1,gid://shopify/Customer/6864981754145,Russell,Winfield,russel.winfield@example.com,en,2023-03-10T18:11:14Z,2023-03-10T18:11:23Z,4,6656.19,USD
2,gid://shopify/Customer/6864981786913,Karine,Ruby,karine.ruby@example.com,en,2023-03-10T18:11:14Z,2023-03-10T18:11:23Z,1,752.18,USD


In [283]:
# clean and create dataframe
def product_dataframe(data2):
  d = data2['data']['productVariants']['edges']
  a = pd.json_normalize(d)
  lista = list(a.columns)
  cols = list(map(lambda x: x.replace('node.', ''), lista))
  a.columns = cols
  product_df = a.copy()
  return product_df

In [284]:
product_dataframe(data2)

,id,displayName,title,sku,inventoryQuantity,price,inventoryItem.requiresShipping
0,gid://shopify/ProductVariant/44692793524513,a - S,S,A123,20,1.00,True
1,gid://shopify/ProductVariant/44692793557281,a - M,M,A124,11,1.00,True
2,gid://shopify/ProductVariant/44692793590049,a - L,L,A125,11,1.00,True
3,gid://shopify/ProductVariant/44692812661025,b - Yellow,Yellow,B223,40,2.00,True
4,gid://shopify/ProductVariant/44692812693793,b - Green,Green,B224,21,2.00,True
5,gid://shopify/ProductVariant/44692828389665,c - S,S,C334,60,3.00,True
6,gid://shopify/ProductVariant/44692828422433,c - M,M,C335,31,3.00,True
7,gid://shopify/ProductVariant/44692828455201,c - L,L,C336,31,3.00,True
8,gid://shopify/ProductVariant/44692852834593,d - Black,Black,D434,80,4.00,True
9,gid://shopify/ProductVariant/44692852867361,d - Gold,Gold,D435,40,4.00,True


# Export Spreadsheets

In [290]:
# export spreadsheets
product_file = product_dataframe(data2)
product_file = product_file.to_csv('product_file.csv', index=False)

In [291]:
customers_file = fetch_bulk(status)
customers_file = customers_file.to_csv('customers_file.csv', index=False)

[Shopify Python API](https://github.com/Shopify/shopify_python_api)